In [ ]:
import langchain

from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os

In [ ]:
with open('../OPENAI_API_KEY', 'r') as file:
    env_value = file.read().strip()

In [ ]:
os.environ['OPENAI_API_KEY'] = env_value

In [ ]:
llm = ChatOpenAI()

In [ ]:
model = ChatOpenAI(model_name="gpt-3.5")

In [ ]:
# llm.invoke("Explain concept of derivatives in mathematics for kid?")

In [ ]:
embeddings = OpenAIEmbeddings()
output_parser = StrOutputParser()
filepath = "STP 420 spring 2024 course syllabus" #file as input
pages = "" #extracting file content here
def load_content(filepath):
    global pages
    path = filepath+".pdf"
    loader = PyPDFLoader(path) 
    pages = loader.load_and_split() #loading file content as document with page content
    db = FAISS.from_documents(pages, embeddings)
    return db
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
# docs = text_splitter.split_documents(pages)



In [ ]:
def get_response_from_query(db, query, k = 4):
    docs = db.similarity_search(query, k=2)
    docs_page_content = " ".join([d.page_content for d in docs])
    template="You are the world renowned problem solver and useful teacher who can answer questions based on course content: {docs}"
    prompt = ChatPromptTemplate.from_messages([("system",template), ("user", query)])
    chain = prompt | llm | output_parser
    return chain.invoke({"input": query, "docs": docs_page_content, "query": query})
db =  load_content(filepath)
response = get_response_from_query(db, "when will we learn confidence interval?")
print(response)